<a href="https://colab.research.google.com/github/HuyenTrang457/MCM/blob/main/model2_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

tải các thư viện cần thiết

In [ ]:
!pip uninstall -y scikit-learn
!pip install scikit-learn==1.3.1
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss

sử dụng bộ dữ liệu lấy từ phần Data-processing, sau đó chọn ra các cột cần thiết là 'p1_games','point_victor','p1_ace','p1_winner','p1_unf_err' 'p1_distance_run', 'rally_count', 'speed_mph','p1_net_pt_won'. Sau đó tách bộ dữ liệu thành 30 bộ dữ liệu nhỏ ứng với từng match_id, lưu thành file csv mới với tên trùng với match_id


In [ ]:
# bổ sung code tách dữ liệu
import pandas as pd
import os

# Load the processed dataset
data_path = "processed_data.csv"  # Replace with your actual processed data file path
df = pd.read_csv(data_path)

# Select necessary columns
columns_to_select = ['match_id', 'p1_games', 'point_victor', 'p1_ace', 'p1_winner', 'p1_unf_err',
                     'p1_distance_run', 'rally_count', 'speed_mph', 'p1_net_pt_won']
df = df[columns_to_select]

# Create a directory to save individual match files
output_dir = "match_files"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Split the dataset by match_id and save each as a separate CSV file
unique_match_ids = df['match_id'].unique()

for match_id in unique_match_ids:
    match_data = df[df['match_id'] == match_id]
    match_file_name = f"{match_id}.csv"
    match_file_path = os.path.join(output_dir, match_file_name)
    match_data.to_csv(match_file_path, index=False)

print(f"Files saved in the folder: {output_dir}")


In [ ]:
# Đường dẫn tới thư mục chứa các file dataset
data_dir = '/content/'

# Liệt kê các file trong thư mục (điều chỉnh tên file theo cấu trúc thực tế)
file_list = [
    '2023-wimbledon-1301.csv',
    '2023-wimbledon-1302.csv',
    '2023-wimbledon-1303.csv',
    '2023-wimbledon-1304.csv',
    '2023-wimbledon-1306.csv',
    '2023-wimbledon-1307.csv',
    '2023-wimbledon-1308.csv',
    '2023-wimbledon-1310.csv',
    '2023-wimbledon-1311.csv',
    '2023-wimbledon-1312.csv',
    '2023-wimbledon-1313.csv',
    '2023-wimbledon-1314.csv',
    '2023-wimbledon-1315.csv',
    '2023-wimbledon-1316.csv',
    '2023-wimbledon-1401.csv',
    '2023-wimbledon-1402.csv',
    '2023-wimbledon-1403.csv',
    '2023-wimbledon-1404.csv',
    '2023-wimbledon-1405.csv',
    '2023-wimbledon-1406.csv',
    '2023-wimbledon-1407.csv',
    '2023-wimbledon-1408.csv',
    '2023-wimbledon-1501.csv',
    '2023-wimbledon-1502.csv',
    '2023-wimbledon-1503.csv',
    '2023-wimbledon-1504.csv',
    '2023-wimbledon-1601.csv',
    '2023-wimbledon-1602.csv',
    '2023-wimbledon-1701.csv'
]




**Part 1: Import Libraries and Load Data**


In [ ]:
###
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, BaggingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss, precision_score, recall_score, f1_score, roc_auc_score, brier_score_loss
import os

**load dữ liệu và chuẩn bị bộ dữ liệu để test model**

Chia dữ liệu thành hai phần:

  Tập huấn luyện (X_train, y_train): Dùng để huấn luyện mô hình (80% dữ liệu).

  Tập kiểm tra (X_test, y_test): Dùng để kiểm tra hiệu quả của mô hình (20% dữ  
    liệu).


In [ ]:
# Load the processed dataset
data_path = "processed_data.csv"  # Replace with your actual processed data file path
df = pd.read_csv(data_path)

 # Split into train and test sets (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


**Part 3: Huấn luyện từng mô hình**

Dùng tập huấn luyện để "dạy" các mô hình dự đoán kết quả


*   Duyệt qua từng mô hình trong danh sách models.
* Đánh giá từng mô hình: Đo lường hiệu quả của từng mô hình qua các chỉ số:

Accuracy: Tỷ lệ dự đoán đúng.

Log Loss: Đo độ chính xác của xác suất dự đoán.

Precision: Độ chính xác của các dự đoán thắng.

Recall: Tỷ lệ tìm đúng các trường hợp thắng thực tế.

F1-Score: Trung bình hài hòa giữa Precision và Recall.

ROC-AUC: Diện tích dưới đường cong ROC.

Brier Score: Độ sai lệch của xác suất dự đoán.


In [ ]:
### Part 3
    # Initialize models
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "Gradient Boosting": GradientBoostingClassifier(random_state=42),
        "SVM": SVC(probability=True, random_state=42),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "kNN": KNeighborsClassifier(),
        "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    }

    results = {}

    # Train and evaluate each model
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred_proba = model.predict_proba(X_test)[:, 1]
        y_pred = (y_pred_proba > 0.5).astype(int)

        # Evaluate model
        accuracy = accuracy_score(y_test, y_pred)
        logloss = log_loss(y_test, y_pred_proba)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_pred_proba)
        brier = brier_score_loss(y_test, y_pred_proba)

        # Save results
        results[name] = {
            "Accuracy": accuracy,
            "Log Loss": logloss,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1,
            "ROC-AUC": roc_auc,
            "Brier Score": brier
        }


In [ ]:
### Part 4: Ensemble Methods
    # Voting Classifier
    voting_clf = VotingClassifier(
        estimators=[
            ('lr', models['Logistic Regression']),
            ('rf', models['Random Forest']),
            ('gb', models['Gradient Boosting']),
            ('svm', models['SVM'])
        ],
        voting='soft'
    )
    voting_clf.fit(X_train, y_train)
    y_pred_proba = voting_clf.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    logloss = log_loss(y_test, y_pred_proba)
    results['Voting Classifier'] = {"Accuracy": accuracy, "Log Loss": logloss}

    # Bagging Classifier
    bagging_clf = BaggingClassifier(estimator=models['Decision Tree'], n_estimators=50, random_state=42)
    bagging_clf.fit(X_train, y_train)
    y_pred_proba = bagging_clf.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    logloss = log_loss(y_test, y_pred_proba)
    results['Bagging'] = {"Accuracy": accuracy, "Log Loss": logloss}

    # Boosting (Gradient Boosting)
    boosting_clf = models['Gradient Boosting']
    boosting_clf.fit(X_train, y_train)
    y_pred_proba = boosting_clf.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    logloss = log_loss(y_test, y_pred_proba)
    results['Boosting'] = {"Accuracy": accuracy, "Log Loss": logloss}

    # Stacking Classifier
    stacking_clf = StackingClassifier(
        estimators=[
            ('lr', models['Logistic Regression']),
            ('rf', models['Random Forest']),
            ('gb', models['Gradient Boosting']),
            ('svm', models['SVM'])
        ],
        final_estimator=models['XGBoost']
    )
    stacking_clf.fit(X_train, y_train)
    y_pred_proba = stacking_clf.predict_proba(X_test)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)
    logloss = log_loss(y_test, y_pred_proba)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    brier = brier_score_loss(y_test, y_pred_proba)

    # Save to results
    results['Stacking'] = {
        "Accuracy": accuracy,
        "Log Loss": logloss,
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "ROC-AUC": roc_auc,
        "Brier Score": brier
    }




**Part 5: Output Results bold text**

In [ ]:
    # Convert results to DataFrame
    results_df = pd.DataFrame(results).T

    # Sort by Accuracy
    results_df = results_df.sort_values(by="Accuracy", ascending=False)

    # Print sorted results
    print(results_df)

else:
    print("No data available after filtering!")